In [1]:
# !which jupyter

In [2]:
# !pip install ultralytics
# !pip install ipywidgets

In [3]:
# %matplotlib widget
# import ipympl
# %matplotlib inline
from ultralytics import YOLO
import PIL
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Image, display, clear_output
import numpy as np
# from scipy.signal import find_peaks
import glob
import os

In [4]:
HOME = os.getcwd()
print(HOME)

/home/nicolas/Documents/counter_Yolo


# Load model

In [5]:
# Load a pretrained model
# model_path = r'C:\Users\nimod\Documents\Simonin_Doc\counter_Yolo\model_v2\v2.1_25_epochs\last.pt' # windows
model_path = r'/home/nicolas/Documents/counter_Yolo/model_v2/v2.1_25_epochs/last.pt'
model = YOLO(model_path)

In [6]:
# print(model.info)

# Crop and select image

In [7]:
List_img = glob.glob(f'./raw_images/*.JPG')
print(List_img)

['./raw_images/IMG_0282.JPG', './raw_images/IMG_0316.JPG', './raw_images/IMG_0301.JPG', './raw_images/IMG_0298.JPG', './raw_images/IMG_0287.JPG', './raw_images/IMG_0290.JPG', './raw_images/IMG_0286.JPG', './raw_images/IMG_0299.JPG', './raw_images/IMG_0294.JPG', './raw_images/IMG_0319.JPG', './raw_images/IMG_0293.JPG', './raw_images/IMG_0318.JPG', './raw_images/IMG_0289.JPG', './raw_images/IMG_0317.JPG', './raw_images/IMG_0285.JPG', './raw_images/IMG_0300.JPG', './raw_images/IMG_0288.JPG', './raw_images/IMG_0304.JPG', './raw_images/IMG_0315.JPG', './raw_images/IMG_0281.JPG', './raw_images/IMG_0303.JPG', './raw_images/IMG_0283.JPG', './raw_images/IMG_0302.JPG', './raw_images/IMG_0292.JPG', './raw_images/IMG_0297.JPG', './raw_images/IMG_0295.JPG', './raw_images/IMG_0284.JPG', './raw_images/IMG_0309.JPG', './raw_images/IMG_0296.JPG', './raw_images/IMG_0308.JPG']


In [8]:
# im = cv2.cvtColor((cv2.imread(List_img[0])), cv2.COLOR_BGR2RGB)

In [9]:
# plot_crop_lines(List_img[0],v_min=0, v_max=0, h_min=200, h_max=300)

In [10]:
# f= List_img[0]
# im = Image.open(f)
# im2arr = np.array(im)
# img = Image.fromarray(im2arr).resize((200,200))
# display(img)

In [11]:
# Avec PIL
img = np.array(PIL.Image.open(List_img[0]))
print(img.shape)
# Avec CV2
# img= cv2.cvtColor(cv2.imread(List_img[0]), cv2.COLOR_BGR2RGB)
# print(img.shape)
# target_ratio = None
# display(PIL.Image.fromarray(img).resize((400,400)),PIL.Image.NEAREST)

(3024, 4032, 3)


In [12]:
image = widgets.Dropdown(
    options=List_img,
    value=List_img[0],
    description='Choix image',)

vert_x_min = widgets.IntSlider(value=100,# int(img.shape[1]/10),
                               min=0,max=1000,#int(img.shape[1]/2),step=25,
                               description='Vert min:',)
vert_x_max = widgets.IntSlider(value=2500, #int(img.shape[1]*9/10),
                               min=0, #int(img.shape[1]/2),
                               max=5000,# img.shape[1],step=25,
                               description='Vert max:',)
hori_y_min = widgets.IntSlider(value=100,#int(img.shape[0]/10),
                               min=0,max=5000,#int(img.shape[0]/2),step=25,
                               description='Hori min:', orientation='Horizontal')
hori_y_max = widgets.IntSlider(value=2500,#int(img.shape[0]*9/10),
                               min=0,#int(img.shape[0]/2),
                               max=5000,#img.shape[0],step=25,
                               description='Hori max:', orientation='Horizontal')

def ratio(f,v_min, v_max, h_min, h_max):
    size_to_display = 501
    print(np.array(PIL.Image.open(f)).shape[1]//size_to_display )
    return np.array(PIL.Image.open(f)).shape[1]//size_to_display 

def plot_crop_lines(f,v_min, v_max, h_min, h_max):
    # global target_ratio
    # Avec matplotlib
    # i = cv2.imread(f)
    # img = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    # height, width, _ = img.shape
    # fig = plt.figure(figsize=(10,10))
    # plt.plot([v_min,v_min], [0,height], color='r')
    # plt.plot([v_max,v_max], [0,height], color='r')
    # plt.plot([0,width], [h_min,h_min], color='r')
    # plt.plot([0,width], [h_max,h_max], color='r')
    # plt.imshow(img)
    # fig, ax = plt.subplots(figsize=(10,10))
    # ax.imshow(img)
    # ax.plot([v_min,v_min], [0,height], color='r')
    # ax.plot([v_max,v_max], [0,height], color='r')
    # ax.plot([0,width], [h_min,h_min], color='r')
    # ax.plot([0,width], [h_max,h_max], color='r')
    # plt.show(fig)
    # plt.close()
    # ax.axis('off')
    # fig.canvas.draw()
    # display(fig)
    # plt.close(fig)
    # plt.show()
    # im = img[vert_x_min.value:vert_x_max.value, hori_y_min.value:hori_y_max.value, :]    
    #
    # Avec Image PIL Et Numpy Array - FONCTIONNE OK
    width = 5
    img_ = np.array(PIL.Image.open(f))
    img_[:, v_min-width:v_min+width, :] = [255, 0, 0]
    img_[:, v_max-width:v_max+width, :] = [255, 0, 0]
    img_[ h_min-width: h_min+width,: , :] = [255, 0, 0]
    img_[ h_max-width: h_max+width,: , :] = [255, 0, 0]
    # im = img[hori_y_min.value:hori_y_max.value, vert_x_min.value:vert_x_max.value, :]
    target_rati = ratio(f,v_min, v_max, h_min, h_max)
    image_display = PIL.Image.fromarray(img_)#.resize((img.shape[1]//target_ratio, img.shape[0]//target_ratio), PIL.Image.NEAREST)
    display(image_display.resize((img_.shape[1]//target_rati,img_.shape[0]//target_rati),PIL.Image.NEAREST ))
    # im = img_[h_min:h_max, v_min:v_max, :]

ui = widgets.HBox([vert_x_min,vert_x_max,hori_y_min,hori_y_max])

graph = widgets.interactive_output(plot_crop_lines,{'f':image, 'v_min':vert_x_min, 
                                                    'v_max':vert_x_max, 'h_min': hori_y_min, 'h_max':hori_y_max})
display(image,ui,graph)    

Dropdown(description='Choix image', options=('./raw_images/IMG_0282.JPG', './raw_images/IMG_0316.JPG', './raw_…

Output()

In [13]:
def show(results):
    for r in results:      
        
        # plt.figure(figsize=(14,5))
        # plt.suptitle('Comptage de planches')

        # plt.subplot(1,2,1)
        # im_array = r.plot(line_width=3)  # plot a BGR numpy array of predictions
        # plt.imshow(im_array) #np.flip(im_array, axis=-1) to correct colors
        # plt.text(0, -50, f"Nombre de planches: {r.__len__()}")
        
        # # On crée le np.array col0à3 coordonnées de la box, 4 et 5 milieu de chaque box, 6:confidence level
        # coord = np.array(r.boxes.xyxy)
        # coord = np.column_stack((coord, (coord[:,0]+coord[:,2])/2, (coord[:,1]+coord[:,3])/2, np.array(r.boxes.conf)))
        # for i,c in enumerate(coord):
        #     plt.text(x=c[4], y=c[5], s=i+1)

        # plt.subplot(1,2,2)
        # plt.plot(coord[:,6], marker='o')
        # plt.xlabel('numéro de box')
        # plt.ylabel('Confidence')
        # plt.grid()
        # plt.show()
        # # plt.save('./predict_image.JPG')

        im_array = r.plot(line_width=3)  # plot a BGR numpy array of predictions
        # im = Image.fromarray(im_array)  # RGB PIL image
        rate = ratio(image.value,vert_x_min.value,vert_x_max.value, hori_y_min.value, hori_y_max.value)
        display(PIL.Image.fromarray(im_array).resize((im_array.shape[1]//rate,im_array.shape[0]//rate),PIL.Image.NEAREST ))
        # im.show()  # show image
        # im.save('results.jpg')  # save im
        
def count_board(conf):
    # Run inference on source
    img_cropped = np.array(PIL.Image.open(image.value))[hori_y_min.value:hori_y_max.value, vert_x_min.value:vert_x_max.value, :]
    results = model.predict(source=img_cropped, # image_to_predict
                            show=False, 
                            conf=conf,
                            save=False, 
                            line_width=3,
                            project='Board_counter',
                            );
    show(results)

c = widgets.FloatSlider(
    value=0.3,
    min=0,max=1,step=0.05,
    description='Seuil de détection:',)

button = widgets.Button(description="Count!")
output = widgets.Output()

# display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        count_board(c.value)

button.on_click(on_button_clicked)

# out = widgets.interactive_output(count_board,{'conf':c})
display(c,button,output)

FloatSlider(value=0.3, description='Seuil de détection:', max=1.0, step=0.05)

Button(description='Count!', style=ButtonStyle())

Output()